In [ ]:
def CheckRange(Start,End,date):
    return int(date) in range(int(Start),int(End))

def FromDayToService(date):
    #date est un tuple ('20210907','tuesday')
    ListServiceId=[]

    for i in range(len(calendardf)):

        startdate,endate=calendardf.iloc[i,[8,9]]
        
        if CheckRange(startdate,endate+1,date[0]):

            if calendardf.loc[i,date[1]]==1:
                
                ListServiceId.append(str(calendardf.iloc[i,0]))

    return ListServiceId

def FromLineIdtoRouteId(LineId):
    routeId = routes[routes['route_short_name']==LineId]['route_id'].iloc[0]
    return routeId

def SelectInGTFS(day,lineId,stopid):
    ListOfService = FromDayToService(day)
    RouteId = FromLineIdtoRouteId(lineId)
    Comparaisondf = stop_times[(stop_times['stop_id']==stopid)&(stop_times['route_id']==RouteId)&(stop_times['service_id'].isin(ListOfService))].sort_values('arrival_time')
    Comparaisondf=Comparaisondf[['trip_id','arrival_time','stop_id','route_id','service_id']]
    ArrivalTime = Comparaisondf['arrival_time'].values
    IntervalTime = list(ArrivalTime[1:]-ArrivalTime[:-1])
    IntervalTime.insert(0,ArrivalTime[0])
    Comparaisondf['Interval']= IntervalTime
    Comparaisondf['Ponctuality']=np.array(IntervalTime)>720
    return Comparaisondf

def ScheduledTime(lineId,stopId,date):
    ListOfService = FromDayToService(date)
    RouteId = FromLineIdtoRouteId(lineId)
    Comparaisondf = stop_times[(stop_times['stop_id']==stopId)&(stop_times['route_id']==RouteId)&(stop_times['service_id'].isin(ListOfService))].sort_values('arrival_time')
    ArrivalTime = Comparaisondf['arrival_time'].values
    IntervalTime = list(ArrivalTime[1:]-ArrivalTime[:-1])
    IntervalTime.insert(0,ArrivalTime[0])
    Punctuality = np.array(IntervalTime)>720 
    IntervalTime = list(zip(IntervalTime,Punctuality))

    return IntervalTime



In [ ]:
Date=('20210907','tuesday')
lineid = '4'
stopid = '5026'


test = SelectInGTFS(Date,lineid,stopid)
test.head(30)
